In [4]:
# train_swin_tiny_stage0_5_v2.py
import os
import random
import glob
from typing import List

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import cv2


In [2]:
# ------------------------
# Device & Seed
# ------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} ({torch.cuda.get_device_name(0) if device.type=='cuda' else 'CPU'})")

# ------------------------
# Paths
# ------------------------
DATA_DIR = "/data/ephemeral/home/data/processed/stage0_5_train/"
META_PATH = "/data/ephemeral/home/data/meta_stage0_5_train.csv"
TRAIN_CSV = "/data/ephemeral/home/data/raw/train.csv"
TEST_PATH = "/data/ephemeral/home/data/processed/stage0_5_test/"
SUB_PATH = "/data/ephemeral/home/data/raw/sample_submission.csv"

# ------------------------
# Config
# ------------------------
model_name = "swin_tiny_patch4_window7_224"
IMG_SIZE = 224
NUM_CLASSES = 17
LR = 3e-4           # v2: 3e-4
EPOCHS = 30         # v2: 30 epochs
BATCH_SIZE = 64
num_workers = 8
warmup_epochs = 3
mixup_alpha = 0.2   # v2: Mixup/CutMix 0.2
cutmix_alpha = 0.2
mixup_prob = 0.2



Using cuda (NVIDIA GeForce RTX 3090)


In [5]:
# ------------------------
# Dataset
# ------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row["filepath"]).convert("RGB"))
        target = int(row["target"])
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target

# ------------------------
# Albumentations Transforms (v2+)
# ------------------------
trn_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.OneOf([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
    ], p=0.6),
    A.Rotate(limit=25, border_mode=cv2.BORDER_REFLECT_101, p=0.4),
    A.ShiftScaleRotate(shift_limit=0.06, scale_limit=0.08, rotate_limit=10, p=0.5),
    A.Perspective(scale=(0.08, 0.12), p=0.2),
    A.ElasticTransform(alpha=20, sigma=5, alpha_affine=10, p=0.15),
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=0.5),
        A.GaussianBlur(blur_limit=3, p=0.5),
        A.GaussNoise(var_limit=(5, 25), p=0.5),
    ], p=0.3),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
        A.RandomShadow(p=0.3),
        A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=0.3),
    ], p=0.4),
    A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=10, val_shift_limit=10, p=0.25),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

tst_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# ------------------------
# Merge meta with train.csv safely
# ------------------------
meta = pd.read_csv(META_PATH)
train_csv = pd.read_csv(TRAIN_CSV)

# Ensure basename column exists; meta saved without group prefix for v7+
if "basename" not in meta.columns:
    meta["basename"] = meta["filepath"].apply(lambda x: os.path.basename(x).split("_", 1)[-1])

train_csv["basename"] = train_csv["ID"].apply(lambda x: os.path.basename(x))
meta_joined = pd.merge(meta, train_csv[["basename", "target"]], on="basename", how="left")
nan_cnt = meta_joined["target"].isna().sum()
if nan_cnt > 0:
    print(f"Warning: dropping {nan_cnt} rows with NaN targets after merge.")
meta_joined = meta_joined.dropna(subset=["target"]).reset_index(drop=True)
meta_joined["target"] = meta_joined["target"].astype(int)

# Split (stratified)
trn_df, val_df = train_test_split(
    meta_joined, test_size=0.2, stratify=meta_joined["target"], random_state=SEED
)
print("Split:", trn_df.shape, val_df.shape)



Split: (1256, 4) (314, 4)


/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_327570/1491902118.py:32: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=20, sigma=5, alpha_affine=10, p=0.15),
/tmp/ipykernel_327570/1491902118.py:36: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 25), p=0.5),


In [6]:
# ------------------------
# Dataloaders
# ------------------------
trn_loader = DataLoader(ImageDataset(trn_df, trn_transform),
                        batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(ImageDataset(val_df, tst_transform),
                        batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=True)

# ------------------------
# Model / Loss / Optim / Scheduler / Mixup
# ------------------------
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=NUM_CLASSES,
    in_chans=3,
    drop_path_rate=0.1
).to(device)

# Use SoftTargetCrossEntropy when Mixup is on
from timm.loss import SoftTargetCrossEntropy
from timm.data.mixup import Mixup

mixup_fn = Mixup(
    mixup_alpha=mixup_alpha, cutmix_alpha=cutmix_alpha,
    cutmix_minmax=None, prob=mixup_prob, switch_prob=0.0, mode='batch',
    label_smoothing=0.0, num_classes=NUM_CLASSES
)

ce_loss = nn.CrossEntropyLoss(label_smoothing=0.1)  # for no-mixup cases
soft_loss = SoftTargetCrossEntropy()

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# Simple warmup + cosine schedule
def cosine_warmup_lr(it, total_it, warmup_it, base_lr):
    if it < warmup_it:
        return base_lr * (it + 1) / warmup_it
    t = (it - warmup_it) / (total_it - warmup_it)
    return base_lr * 0.5 * (1 + np.cos(np.pi * t))

total_iters = EPOCHS * len(trn_loader)
warmup_iters = max(1, warmup_epochs * len(trn_loader))

scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))



/tmp/ipykernel_327570/3536936732.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))


In [7]:
# ------------------------
# Train / Validate
# ------------------------
def train_one_epoch(epoch):
    model.train()
    total_loss, preds_list, targets_list = 0.0, [], []
    it_base = epoch * len(trn_loader)

    for it, (images, targets) in enumerate(tqdm(trn_loader, desc=f"Train {epoch+1}")):
        global_it = it_base + it
        lr = cosine_warmup_lr(global_it, total_iters, warmup_iters, LR)
        for pg in optimizer.param_groups:
            pg["lr"] = lr

        images = images.to(device)
        targets = targets.to(device)

        use_mix = (mixup_prob > 0)
        if use_mix:
            images, targets_mix = mixup_fn(images, targets)
        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            logits = model(images)
            if use_mix:
                loss = soft_loss(logits, targets_mix)
            else:
                loss = ce_loss(logits, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        # metrics (argmax uses hard labels even when mixup on)
        preds = logits.detach().argmax(1).cpu().numpy()
        preds_list.extend(preds)
        targets_list.extend(targets.detach().cpu().numpy())

    metrics = {
        "train_loss": total_loss / len(trn_loader),
        "train_acc": accuracy_score(targets_list, preds_list),
        "train_f1": f1_score(targets_list, preds_list, average="macro"),
    }
    return metrics

@torch.no_grad()
def validate():
    model.eval()
    total_loss, preds_list, targets_list = 0.0, [], []
    for images, targets in tqdm(val_loader, desc="Valid"):
        images = images.to(device)
        targets = targets.to(device)
        with torch.amp.autocast("cuda"):
            logits = model(images)
            loss = ce_loss(logits, targets)
        total_loss += loss.item()
        preds_list.extend(logits.argmax(1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
    return {
        "val_loss": total_loss / len(val_loader),
        "val_acc": accuracy_score(targets_list, preds_list),
        "val_f1": f1_score(targets_list, preds_list, average="macro"),
    }



In [8]:
# ------------------------
# Training Loop
# ------------------------
best_f1 = 0.0
for epoch in range(EPOCHS):
    trn_m = train_one_epoch(epoch)
    val_m = validate()

    print(f"\n[Epoch {epoch+1}/{EPOCHS}] "
          f"Train | Loss {trn_m['train_loss']:.4f} Acc {trn_m['train_acc']:.4f} F1 {trn_m['train_f1']:.4f} | "
          f"Valid | Loss {val_m['val_loss']:.4f} Acc {val_m['val_acc']:.4f} F1 {val_m['val_f1']:.4f}")

    if val_m["val_f1"] > best_f1:
        best_f1 = val_m["val_f1"]
        torch.save(model.state_dict(), f"./best_swin_tiny_v2_f1_{best_f1:.4f}.pt")
        print(f"Saved best checkpoint (F1={best_f1:.4f})")



Train 1:   0%|                                                                                                           | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_327570/2255632436.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.53it/s]



[Epoch 1/30] Train | Loss 2.5085 Acc 0.2229 F1 0.2029 | Valid | Loss 1.6800 Acc 0.5860 F1 0.5024
Saved best checkpoint (F1=0.5024)


Train 2:   0%|                                                                                                           | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_327570/2255632436.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.97it/s]



[Epoch 2/30] Train | Loss 1.2279 Acc 0.5334 F1 0.4980 | Valid | Loss 1.1594 Acc 0.8344 F1 0.8144
Saved best checkpoint (F1=0.8144)


Train 3:   0%|                                                                                                           | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_327570/2255632436.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.56it/s]


...


[Epoch 28/30] Train | Loss 0.1148 Acc 0.9761 F1 0.9746 | Valid | Loss 1.0763 Acc 0.9204 F1 0.9174


Train 29:   0%|                                                                                                          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_327570/2255632436.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.45it/s]



[Epoch 29/30] Train | Loss 0.1458 Acc 0.9132 F1 0.9075 | Valid | Loss 1.0803 Acc 0.9268 F1 0.9226


Train 30:   0%|                                                                                                          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_327570/2255632436.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.81it/s]


[Epoch 30/30] Train | Loss 0.1482 Acc 0.7970 F1 0.7905 | Valid | Loss 1.0807 Acc 0.9236 F1 0.9200


In [9]:
# ------------------------
# Inference with light TTA
# ------------------------
print("\nInference...")
model.load_state_dict(torch.load(f"./best_swin_tiny_v2_f1_{best_f1:.4f}.pt", map_location=device))
model.eval()

sub = pd.read_csv(SUB_PATH)

# build file index from processed test
test_files = glob.glob(os.path.join(TEST_PATH, "**", "*.*"), recursive=True)
idx = {os.path.basename(p): p for p in test_files}

def tta_predict(img_np: np.ndarray) -> np.ndarray:
    # original
    ims: List[np.ndarray] = [img_np]
    # flips and small rotations
    ims.append(np.ascontiguousarray(np.flip(img_np, axis=1)))  # hflip
    ims.append(np.ascontiguousarray(np.flip(img_np, axis=0)))  # vflip

    logits_sum = None
    for im in ims:
        tens = tst_transform(image=im)["image"].unsqueeze(0).to(device)
        with torch.no_grad():
            logit = model(tens)
        logits_sum = logit if logits_sum is None else (logits_sum + logit)
    return (logits_sum / len(ims)).softmax(dim=1).cpu().numpy()

preds = []
missing = 0
for name in tqdm(sub["ID"], desc="TTA Inference"):
    p = idx.get(name, None)
    if p is None:
        missing += 1
        raise FileNotFoundError(f"Missing test image: {name}")
    img_np = np.array(Image.open(p).convert("RGB"))
    prob = tta_predict(img_np)
    preds.append(int(prob.argmax(axis=1)[0]))

if missing:
    print(f"Warning: {missing} test files were missing.")

sub["target"] = preds
sub.to_csv("v2_swin_tiny_split.csv", index=False)
print("Done. Saved to v2_swin_tiny_split.csv")



Inference...


TTA Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████| 3140/3140 [01:50<00:00, 28.49it/s]

Done. Saved to v2_swin_tiny_split.csv
